<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/ACP_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GEMINI

In [ ]:
!pip install google-generativeai -q
!pip install airportsdata -q
!pip install colab-env -q
import colab_env
import os

In [ ]:
# Used to securely store your API key
from google.colab import userdata

import pathlib
import textwrap

import google.generativeai as genai

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import airportsdata

# Set your API key using userdata
GOOGLE_API_KEY = userdata.get('GEMINI')
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize Gemini 2.0
model_name = 'models/gemini-2.0-flash'
model = genai.GenerativeModel(model_name)

# Function to get airport geolocation (using airportsdata module)
def get_airport_geolocation(airport_code):
    airports = airportsdata.load('ICAO')
    airport = airports.get(airport_code)
    if airport:
        return (airport['lat'], airport['lon'])
    else:
        return None

class FlightPlanningAgent:
    def __init__(self, model):
        self.model = model
        # Improved prompt for ACP with LAX as departure for Flight 2
        self.flight_plan_prompt = """
        You are an AI flight planner and dispatcher/controller.
        I'll give you the details of two flights, and you'll generate detailed flight plans that are suitable for Aircraft Pairing (ACP). You'll also coordinate with air traffic control to ensure the safe and efficient execution of these flights.

        Here's the flight information:
        Departure Airport 1: {departure_airport1} ({departure_airport_geolocation1})
        Arrival Airport 1: {arrival_airport1}
        Aircraft Type 1: {aircraft_type1}
        Weather Conditions: {weather_conditions}
        Airspace Restrictions: {airspace_restrictions}
        Other Constraints 1: Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints.

        Departure Airport 2: Please choose a second flight plan departing from the USA, specifically from KLAX (Los Angeles International Airport).
        Arrival Airport 2: {arrival_airport2}
        Aircraft Type 2: Please select the aircraft type as Airbus 350 for the second flight.
        Weather Conditions: {weather_conditions}
        Airspace Restrictions: {airspace_restrictions}
        Other Constraints 2: Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints.

        Pairing Constraints:
        * The two aircraft should follow nearly identical routes, with minimal deviation between them.
        * The difference in cruising altitudes should not exceed 1000 feet.
        * Both aircraft should arrive at the destination within a 30-minute window.

        Lead Aircraft Selection:
        * Please select one of the two aircraft as the lead aircraft for the ACP formation, considering factors such as aircraft performance, fuel efficiency, and route compatibility. Explain the rationale behind your choice.

        Possible Waypoints (ICAO codes):
        * CYYZ (Toronto Pearson International Airport), (43.677, -79.631)
        * CYUL (Montreal Trudeau International Airport), (45.470, -73.741)
        * BIRD (Keflavik International Airport), (63.985, -22.605)
        * EGPF (Glasgow Airport), (55.872, -4.433)
        * ZBAA (Beijing Capital International Airport), (40.079, 116.585)
        * KLAX (Los Angeles International Airport), (33.9425, -118.4081)
        * AWAKEN (N 62°00'00" W 020°00'00")

        Please generate comprehensive flight plans for both flights that meet the ACP criteria. Include the following for each flight:
        * Route: A list of waypoints with their ICAO codes and coordinates, following the format 'KJFK (40.641, -73.985)'.
        * Cruising Altitude: The optimal cruising altitude in feet.
        * Estimated Time of Arrival: The estimated time of arrival in UTC.
        * Fuel Required: The estimated fuel required for the flight in kg.
        * Other Details: Any other relevant details, such as remarks about the route or weather.

        Please provide detailed remarks on the following:
        * Specific weather impact on the route and ETA, including wind conditions and potential for turbulence.
        * Potential airspace restrictions or areas of congestion along the route, and how they will be addressed.
        * Any other factors that might affect ACP compliance, such as aircraft performance differences or potential delays.

        Example Flight Plan (for a single flight):
        Flight Plan: KJFK to LFPG
        Waypoints:
        KJFK (40.641, -73.985)
        CYYZ (43.677, -79.631)
        BIRD (63.985, -22.605)
        EGPF (55.872, -4.433)
        EGLL (51.470, -0.462)
        LFPG (49.009, 2.548)
        Cruising Altitude: 36000 feet
        Estimated Time of Arrival: 2024-10-27T18:00:00Z
        Fuel Required: 7000 kg
        Remarks: None
        """

    def generate_flight_plan(self, departure_airport1, arrival_airport1, aircraft_type1,
                             departure_airport2, arrival_airport2, aircraft_type2,
                             weather_conditions, airspace_restrictions,
                             other_constraints1, other_constraints2):
        # Get geolocations
        departure_geolocation1 = get_airport_geolocation(departure_airport1)
        arrival_geolocation1 = get_airport_geolocation(arrival_airport1)
        departure_geolocation2 = get_airport_geolocation(departure_airport2)
        arrival_geolocation2 = get_airport_geolocation(arrival_airport2)

        if departure_geolocation1 is None or arrival_geolocation1 is None or \
           departure_geolocation2 is None or arrival_geolocation2 is None:
            print(f"Error: Could not find geolocation for one or both airports.")
            return None

        response = self.model.generate_content(
            self.flight_plan_prompt.format(
                departure_airport1=departure_airport1,
                arrival_airport1=arrival_airport1,
                departure_airport_geolocation1=str(departure_geolocation1),
                aircraft_type1=aircraft_type1,
                other_constraints1=other_constraints1,
                departure_airport2=departure_airport2,
                arrival_airport2=arrival_airport2,
                departure_airport_geolocation2=str(departure_geolocation2),
                aircraft_type2=aircraft_type2,
                other_constraints2=other_constraints2,
                weather_conditions=weather_conditions,
                airspace_restrictions=airspace_restrictions
            ),
            generation_config={
                "temperature": 0.1,
                "max_output_tokens": 2048,
            }
        )
        return response.text

# Example Usage
agent = FlightPlanningAgent(model)

# Flight 1 data
departure1 = "CYUL"
arrival1 = "ZSPD"
aircraft1 = "Boeing 777"
constraints1 = "Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints."

# Flight 2 data
departure2 = "KLAX"
arrival2 = "ZSPD"
aircraft2 = "Airbus 350"
constraints2 = "Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints."

# Common data
weather = "Clear skies, light winds"
airspace = "Minimal restrictions"

flight_plan = agent.generate_flight_plan(
    departure_airport1=departure1, arrival_airport1=arrival1, aircraft_type1=aircraft1,
    departure_airport2=departure2, arrival_airport2=arrival2, aircraft_type2=aircraft2,
    weather_conditions=weather, airspace_restrictions=airspace,
    other_constraints1=constraints1, other_constraints2=constraints2
)

print("Generated Flight Plans for ACP:")
print(flight_plan)

Generated Flight Plans for ACP:
Okay, I'm ready to generate the flight plans for the Aircraft Pairing (ACP) operation from CYUL and KLAX to ZSPD. I will prioritize fuel efficiency while adhering to the pairing constraints and considering the provided weather conditions and airspace restrictions.

**Lead Aircraft Selection:**

I will select the **Airbus A350 (KLAX-ZSPD)** as the lead aircraft. Here's the rationale:

*   **Fuel Efficiency:** The Airbus A350 is generally known for its superior fuel efficiency compared to the Boeing 777, especially on long-haul routes. This makes it a better choice to set the pace for fuel conservation.
*   **Route Compatibility:** Both aircraft are capable of flying the great circle route, but the A350's slightly longer range and fuel efficiency give it a slight advantage in maintaining the planned route without needing significant adjustments.
*   **Operational Considerations:** Starting the lead aircraft from KLAX allows for a longer, more direct route 

## openai

In [ ]:
from openai import OpenAI
import os
import colab_env
import airportsdata
import openai

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to get airport geolocation (using airportsdata module)
def get_airport_geolocation(airport_code):
    airports = airportsdata.load('ICAO')
    airport = airports.get(airport_code)
    if airport:
        return (airport['lat'], airport['lon'])
    else:
        return None

class FlightPlanningAgent:
    def __init__(self):
        # No need to initialize a model object here, we'll use openai.ChatCompletion.create directly
        # Improved prompt for ACP with LAX as departure for Flight 2 and Airbus 350 as aircraft type
        self.flight_plan_prompt = """
       You are an AI flight planner and dispatcher/controller.
I'll give you the details of two flights, and you'll generate detailed flight plans that are suitable for Aircraft Pairing (ACP). You'll also coordinate with air traffic control to ensure the safe and efficient execution of these flights.

Here's the flight information:
Departure Airport 1: {departure_airport1} ({departure_airport_geolocation1})
Arrival Airport 1: {arrival_airport1}
Aircraft Type 1: {aircraft_type1}
Weather Conditions: {weather_conditions}
Airspace Restrictions: {airspace_restrictions}
Other Constraints 1: Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints.

Departure Airport 2: Please choose a second flight plan departing from the USA, specifically from KLAX (Los Angeles International Airport).
Arrival Airport 2: {arrival_airport2}
Aircraft Type 2: Please select the aircraft type as Airbus 350 for the second flight.
Weather Conditions: {weather_conditions}
Airspace Restrictions: {airspace_restrictions}
Other Constraints 2: Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints.

Pairing Constraints:
* The two aircraft should follow nearly identical routes, with minimal deviation between them.
* The difference in cruising altitudes should not exceed 1000 feet.
* Both aircraft should arrive at the destination within a 30-minute window.

Lead Aircraft Selection:
* Please select one of the two aircraft as the lead aircraft for the ACP formation, considering factors such as aircraft performance, fuel efficiency, and route compatibility. Explain the rationale behind your choice.

## Possible Waypoints (ICAO codes):
* CYYZ (Toronto Pearson International Airport), (43.677, -79.631)
* CYUL (Montreal Trudeau International Airport), (45.470, -73.741)
* BIRD (Keflavik International Airport), (63.985, -22.605)
* EGPF (Glasgow Airport), (55.872, -4.433)
* ZBAA (Beijing Capital International Airport), (40.079, 116.585)
* KLAX (Los Angeles International Airport), (33.9425, -118.4081)
* AWAKEN (N 62°00'00" W 020°00'00")

Please generate comprehensive flight plans for both flights that meet the ACP criteria. Include the following for each flight:
* Route: A list of waypoints with their ICAO codes and coordinates, following the format 'KJFK (40.641, -73.985)'.
* Cruising Altitude: The optimal cruising altitude in feet.
* Estimated Time of Arrival: The estimated time of arrival in UTC.
* Fuel Required: The estimated fuel required for the flight in kg.
* Other Details: Any other relevant details, such as remarks about the route or weather.

Please provide detailed remarks on the following:
* Specific weather impact on the route and ETA, including wind conditions and potential for turbulence.
* Potential airspace restrictions or areas of congestion along the route, and how they will be addressed.
* Any other factors that might affect ACP compliance, such as aircraft performance differences or potential delays.

## Example Flight Plan (for a single flight):
Flight Plan: KJFK to LFPG
Waypoints:
KJFK (40.641, -73.985)
CYYZ (43.677, -79.631)
BIRD (63.985, -22.605)
EGPF (55.872, -4.433)
EGLL (51.470, -0.462)
LFPG (49.009, 2.548)
Cruising Altitude: 36000 feet
Estimated Time of Arrival: 2024-10-27T18:00:00Z
Fuel Required: 7000 kg
Remarks: None
        """

    def generate_flight_plan(self, departure_airport1, arrival_airport1, aircraft_type1,
                             departure_airport2, arrival_airport2, aircraft_type2,
                             weather_conditions, airspace_restrictions,
                             other_constraints1, other_constraints2):
        # Get geolocations
        departure_geolocation1 = get_airport_geolocation(departure_airport1)
        arrival_geolocation1 = get_airport_geolocation(arrival_airport1)
        departure_geolocation2 = get_airport_geolocation(departure_airport2)
        arrival_geolocation2 = get_airport_geolocation(arrival_airport2)

        if departure_geolocation1 is None or arrival_geolocation1 is None or \
           departure_geolocation2 is None or arrival_geolocation2 is None:
            print(f"Error: Could not find geolocation for one or both airports.")
            return None

        response = openai.chat.completions.create(  # Corrected API call
            model="gpt-4",  # Use GPT-4
            messages=[
                {"role": "system", "content": self.flight_plan_prompt.format(
                    departure_airport1=departure_airport1,
                    arrival_airport1=arrival_airport1,
                    departure_airport_geolocation1=str(departure_geolocation1),
                    aircraft_type1=aircraft_type1,
                    other_constraints1=other_constraints1,
                    departure_airport2=departure_airport2,
                    arrival_airport2=arrival_airport2,
                    departure_airport_geolocation2=str(departure_geolocation2),
                    aircraft_type2=aircraft_type2,
                    other_constraints2=other_constraints2,
                    weather_conditions=weather_conditions,
                    airspace_restrictions=airspace_restrictions
                )}
            ]
        )
        #return response.choices.message['content']  # Access the content correctly
        return response.choices[0].message.content.strip()

# Example Usage
agent = FlightPlanningAgent()

# Flight 1 data
departure1 = "CYUL"
arrival1 = "ZSPD"
aircraft1 = "Boeing 777"
constraints1 = "Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints."

# Flight 2 data
departure2 = "KLAX"
arrival2 = "ZSPD"
aircraft2 = "Airbus 350"
constraints2 = "Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints."

# Common data
weather = "Clear skies, light winds"
airspace = "Minimal restrictions"

flight_plan = agent.generate_flight_plan(
    departure_airport1=departure1, arrival_airport1=arrival1, aircraft_type1=aircraft1,
    departure_airport2=departure2, arrival_airport2=arrival2, aircraft_type2=aircraft2,
    weather_conditions=weather, airspace_restrictions=airspace,
    other_constraints1=constraints1, other_constraints2=constraints2
)

print("Generated Flight Plans for ACP:")
print(flight_plan)

Generated Flight Plans for ACP:
# Flight Plan 1 (Lead Aircraft): Boeing 777 | CYUL to ZSPD

Waypoints:
  * CYUL (45.470, -73.741) -> Departure Airport
  * BIRD (63.985, -22.605)
  * ZBAA (40.079, 116.585)
  * ZSPD (31.1443, 121.8052) -> Arrival Airport

Cruising Altitude: 35,000 feet (Considering fuel efficiency, this lower altitude is chosen compared to traditional commercial flight cruising altitude 36,000- 38,000 feet)

Estimated Time of Arrival: 2024-10-27T19:00:00Z

Fuel Required: Estimate of 121000 kg (Total fuel). The Boeing 777 burns about 6.8 ton of fuel per hour. The approximate flight time CYUL to ZSPD is around 18 hours.

Other Details: Follow Flight Plan 1.

# Flight Plan 2: Airbus 350 | KLAX to ZSPD

Waypoints:
  * KLAX (33.9425, -118.4081) -> Departure Airport
  * AWAKEN (N 62°00'00" W 020°00'00")
  * ZBAA (40.079, 116.585)
  * ZSPD (31.1443, 121.8052) -> Arrival Airport

Cruising Altitude: 34,000 feet (To comply with the ACP)

Estimated Time of Arrival: 2024-10-27T19:30

## CLAUDE 3

In [ ]:
!pip install anthropic -q

!pip install colab-env --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 4.1 MB/s eta 0:00:00


In [ ]:
import anthropic
import os
import colab_env
import json

In [ ]:
def claude_chat(prompt,model,client):
    message = client.messages.create(
    #model="claude-3-opus-20240229",
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": prompt}
    ]
    )
    print()
    print("-" * 80)
    print('Question: %s'%prompt)
    print("-" * 80)
    print()
    print('Answer: ')
    print(message.content[0].text)
    #return message.content[0].text

In [ ]:
api_key = os.environ["CLAUDE3_API_KEY"]
#model="claude-3-opus-20240229"
model="claude-3-5-sonnet-20240620"

client = anthropic.Anthropic(
    api_key=api_key,
)

In [ ]:
message = client.messages.create(
    #model="claude-3-opus-20240229",
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
print(message.content[0].text)

The code uses the Anthropic Claude 3 large language model to generate flight plans, specifically focusing on Aircraft Pairing (ACP) scenarios where two aircraft fly similar routes.

Key Components:

1. get_airport_geolocation(airport_code): This function retrieves the latitude and longitude of an airport using its ICAO code and the airportsdata library.

2. FlightPlanningAgent class:

* __init__(self): Initializes the class and defines a detailed prompt for the Claude 3 model. The prompt provides instructions for generating ACP flight plans, including flight details, constraints, and desired information in the output.

* generate_flight_plan(...): This method orchestrates the flight plan generation process:
  * Gets airport locations using get_airport_geolocation.

  * Sends the prompt to the Claude 3 model via the anthropic library.

  * Receives the generated flight plans from the model.


3. Returns the flight plans as text.
Example Usage:

   * Creates an instance of FlightPlanningAgent.

   * Sets flight parameters for two flights.

   * Calls generate_flight_plan to generate the flight plans.

   * Prints the generated flight plans.

Workflow:

1. The user provides flight details (departure, arrival, aircraft type, etc.) for two flights.

2. The FlightPlanningAgent uses these details to create a structured prompt for Claude 3.

3. The prompt is sent to Claude 3, which generates flight plans based on its knowledge and the provided information.

4. The generated flight plans are returned to the user and printed.


Dependencies:

* anthropic: To interact with the Claude 3 API.

* airportsdata: To retrieve airport information.

* colab_env: Likely for setting up the environment in Google Colab.

* json: Potentially for handling data from the API responses.

Purpose:

The code aims to automate the creation of optimized, ACP-compliant flight plans, taking into account weather, airspace restrictions, and fuel efficiency. This simplifies the flight planning process and potentially leads to safer and more cost-effective flights.

In [ ]:
# Function to get airport geolocation (using airportsdata module)
def get_airport_geolocation(airport_code):
    airports = airportsdata.load('ICAO')
    airport = airports.get(airport_code)
    if airport:
        return (airport['lat'], airport['lon'])
    else:
        return None

class FlightPlanningAgent:
    def __init__(self):
        # No need to initialize a model object here, we'll use openai.ChatCompletion.create directly
        # Improved prompt for ACP with LAX as departure for Flight 2 and Airbus 350 as aircraft type
        self.flight_plan_prompt = """
       You are an AI flight planner and dispatcher/controller.
I'll give you the details of two flights, and you'll generate detailed flight plans that are suitable for Aircraft Pairing (ACP). You'll also coordinate with air traffic control to ensure the safe and efficient execution of these flights.

Here's the flight information:
Departure Airport 1: {departure_airport1} ({departure_airport_geolocation1})
Arrival Airport 1: {arrival_airport1}
Aircraft Type 1: {aircraft_type1}
Weather Conditions: {weather_conditions}
Airspace Restrictions: {airspace_restrictions}
Other Constraints 1: Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints.

Departure Airport 2: Please choose a second flight plan departing from the USA, specifically from KLAX (Los Angeles International Airport).
Arrival Airport 2: {arrival_airport2}
Aircraft Type 2: Please select the aircraft type as Airbus 350 for the second flight.
Weather Conditions: {weather_conditions}
Airspace Restrictions: {airspace_restrictions}
Other Constraints 2: Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints.

Pairing Constraints:
* The two aircraft should follow nearly identical routes, with minimal deviation between them.
* The difference in cruising altitudes should not exceed 1000 feet.
* Both aircraft should arrive at the destination within a 30-minute window.

Lead Aircraft Selection:
* Please select one of the two aircraft as the lead aircraft for the ACP formation, considering factors such as aircraft performance, fuel efficiency, and route compatibility. Explain the rationale behind your choice.

## Possible Waypoints (ICAO codes):
* CYYZ (Toronto Pearson International Airport), (43.677, -79.631)
* CYUL (Montreal Trudeau International Airport), (45.470, -73.741)
* BIRD (Keflavik International Airport), (63.985, -22.605)
* EGPF (Glasgow Airport), (55.872, -4.433)
* ZBAA (Beijing Capital International Airport), (40.079, 116.585)
* KLAX (Los Angeles International Airport), (33.9425, -118.4081)
* AWAKEN (N 62°00'00" W 020°00'00")

Please generate comprehensive flight plans for both flights that meet the ACP criteria. Include the following for each flight:
* Route: A list of waypoints with their ICAO codes and coordinates, following the format 'KJFK (40.641, -73.985)'.
* Cruising Altitude: The optimal cruising altitude in feet.
* Estimated Time of Arrival: The estimated time of arrival in UTC.
* Fuel Required: The estimated fuel required for the flight in kg.
* Other Details: Any other relevant details, such as remarks about the route or weather.

Please provide detailed remarks on the following:
* Specific weather impact on the route and ETA, including wind conditions and potential for turbulence.
* Potential airspace restrictions or areas of congestion along the route, and how they will be addressed.
* Any other factors that might affect ACP compliance, such as aircraft performance differences or potential delays.

## Example Flight Plan (for a single flight):
Flight Plan: KJFK to LFPG
Waypoints:
KJFK (40.641, -73.985)
CYYZ (43.677, -79.631)
BIRD (63.985, -22.605)
EGPF (55.872, -4.433)
EGLL (51.470, -0.462)
LFPG (49.009, 2.548)
Cruising Altitude: 36000 feet
Estimated Time of Arrival: 2024-10-27T18:00:00Z
Fuel Required: 7000 kg
Remarks: None
        """

    def generate_flight_plan(self, departure_airport1, arrival_airport1, aircraft_type1,
                             departure_airport2, arrival_airport2, aircraft_type2,
                             weather_conditions, airspace_restrictions,
                             other_constraints1, other_constraints2):
        # Get geolocations
        departure_geolocation1 = get_airport_geolocation(departure_airport1)
        arrival_geolocation1 = get_airport_geolocation(arrival_airport1)
        departure_geolocation2 = get_airport_geolocation(departure_airport2)
        arrival_geolocation2 = get_airport_geolocation(arrival_airport2)

        if departure_geolocation1 is None or arrival_geolocation1 is None or \
           departure_geolocation2 is None or arrival_geolocation2 is None:
            print(f"Error: Could not find geolocation for one or both airports.")
            return None

        api_key = os.environ["CLAUDE3_API_KEY"]
        #model="claude-3-opus-20240229"
        model="claude-3-5-sonnet-20240620"

        client = anthropic.Anthropic(
            api_key=api_key,
)

        message = client.messages.create(
        #model="claude-3-opus-20240229",
        model="claude-3-5-sonnet-20240620",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": "Hello, Claude"}
        ]
    )

        response = client.messages.create(  # Corrected API call
            model="claude-3-5-sonnet-20240620",
            max_tokens=1024,
            messages=[
                {"role": "user", "content": self.flight_plan_prompt.format(
                    departure_airport1=departure_airport1,
                    arrival_airport1=arrival_airport1,
                    departure_airport_geolocation1=str(departure_geolocation1),
                    aircraft_type1=aircraft_type1,
                    other_constraints1=other_constraints1,
                    departure_airport2=departure_airport2,
                    arrival_airport2=arrival_airport2,
                    departure_airport_geolocation2=str(departure_geolocation2),
                    aircraft_type2=aircraft_type2,
                    other_constraints2=other_constraints2,
                    weather_conditions=weather_conditions,
                    airspace_restrictions=airspace_restrictions
                )}
            ]
        )

        return response.content[0].text

# Example Usage
agent = FlightPlanningAgent()

# Flight 1 data
departure1 = "CYUL"
arrival1 = "ZSPD"
aircraft1 = "Boeing 777"
constraints1 = "Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints."

# Flight 2 data
departure2 = "KLAX"
arrival2 = "ZSPD"
aircraft2 = "Airbus 350"
constraints2 = "Minimize fuel consumption. Prioritize fuel efficiency while adhering to the pairing constraints."

# Common data
weather = "Clear skies, light winds"
airspace = "Minimal restrictions"

flight_plan = agent.generate_flight_plan(
    departure_airport1=departure1, arrival_airport1=arrival1, aircraft_type1=aircraft1,
    departure_airport2=departure2, arrival_airport2=arrival2, aircraft_type2=aircraft2,
    weather_conditions=weather, airspace_restrictions=airspace,
    other_constraints1=constraints1, other_constraints2=constraints2
)

print("Generated Flight Plans for ACP:")
print(flight_plan)

Generated Flight Plans for ACP:
Thank you for providing the flight details. I'll generate comprehensive flight plans for both flights that meet the Aircraft Pairing (ACP) criteria.

Flight Plan 1: CYUL to ZSPD (Boeing 777)

Waypoints:
CYUL (45.4706, -73.7408)
CYYZ (43.677, -79.631)
BIRD (63.985, -22.605)
AWAKEN (62.000, -20.000)
ZBAA (40.079, 116.585)
ZSPD (31.143, 121.805)

Cruising Altitude: 35000 feet
Estimated Time of Arrival: 2024-10-28T02:30:00Z
Fuel Required: 85000 kg
Remarks: Great Circle route with minimal deviations for optimal fuel efficiency.

Flight Plan 2: KLAX to ZSPD (Airbus A350)

Waypoints:
KLAX (33.9425, -118.4081)
AWAKEN (62.000, -20.000)
BIRD (63.985, -22.605)
ZBAA (40.079, 116.585)
ZSPD (31.143, 121.805)

Cruising Altitude: 34000 feet
Estimated Time of Arrival: 2024-10-28T02:45:00Z
Fuel Required: 78000 kg
Remarks: Route optimized to meet ACP constraints with Flight 1.

Lead Aircraft Selection:
I recommend selecting the Boeing 777 (Flight 1) as the lead aircraft fo